In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import csv

## Config

In [83]:
TRAINING_DATA_PATH = './TargetData-TF-Train.csv'
VALIDATION_DATA_PATH = './TargetData-TF-Valid.csv'
TEST_DATA_PATH = './TargetData-TF-Test.csv'

IMAGE_FOLDER_PATH = './TargetImages-TF/'

TARGET_CLASSES_MAPPING = {}
TARGET_CLASSES_MAPPING[10419] = 'Qutub Minar'
TARGET_CLASSES_MAPPING[47378] = 'Eiffel Tower'
TARGET_CLASSES_MAPPING[168098] = 'Golden Gate Bridge'
TARGET_CLASSES_MAPPING[162833] = 'Pakistan Monument'
TARGET_CLASSES_MAPPING[1924] = 'Niagara River'

SIFT_FACTOR = 0.50
ORB_FACTOR = 0.50

orb = cv2.ORB_create()
sift = cv2.xfeatures2d.SIFT_create()

index_params = dict(algorithm=0, trees=5)
search_params = dict()

## Dataset

In [ ]:
def load_data(filename):
    df = pd.read_csv(filename)
    
    id_mapping = {}
    id_sift_des = {}
    id_orb_des = {}
    
    
    for index in range(len(df)):
        image_id = df['id'][index]
        image_label = df['landmark_id'][index]
        
        if index % 25 == 0:
            print(f'Read {index} images')
            
        id_mapping[image_id] = image_label
        image_path = os.path.join(IMAGE_FOLDER_PATH, image_id + '.jpg')
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        _, des_orb = orb.detectAndCompute(image, None)
        _, des_sift = sift.detectAndCompute(image, None)
        
        id_sift_des[image_id] = des_sift
        id_orb_des[image_id] = des_orb
        
    
    return id_mapping, id_sift_des, id_orb_des

In [ ]:
trainid_mapping, trainid_sift_des, trainid_orb_des = load_data(TRAINING_DATA_PATH)

In [ ]:
validid_mapping, validid_sift_des, validid_orb_des = load_data(VALIDATION_DATA_PATH)

In [106]:
def good_matching(matches, factor):    
    good_points = []
    for m, n in matches:
        if m.distance < factor*n.distance:
            good_points.append(m)    
    
    return len(good_points)


def matching(des_a, des_b):
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des_a, des_b, k=2)
   
    return matches


def tf_predictor(referenceid_mapping, referenceid_sift_des, referenceid_orb_des,
                sample_sift_des, sample_orb_des):
    
    good_matches_sift = {}
    good_matches_orb = {}
    for key,values in TARGET_CLASSES_MAPPING.items():
        good_matches_sift[key] = 0
        good_matches_orb[key] = 0
        
    index = 0
    for image_id, image_label in referenceid_mapping.items():
        
        good_matches_sift[image_label] += good_matching(
            matching(referenceid_sift_des[image_id], sample_sift_des),
            SIFT_FACTOR)
        
        index += 1
        if index % 25 == 0:
            #print(f'Referenced : {index} training images')
            pass
        
    max_matches = 0
    max_label = -1
    for image_label, good_match_count in  good_matches_sift.items():
        if (good_match_count + good_matches_orb[image_label]) > max_matches:
            max_matches = good_match_count + good_matches_orb[image_label]
            max_label = image_label
            
    
    return max_label, good_matches_sift, good_matches_orb

In [108]:
correct_count = 0
index = 0

for image_id, image_label in validid_mapping.items():
    predicted_label, _ , _ = tf_predictor(
        trainid_mapping, trainid_sift_des, trainid_sift_des,
        validid_sift_des[image_id], validid_sift_des[image_id]
    )
    
    if predicted_label == image_label:
        correct_count += 1
    
    index += 1
    if index % 1 == 0:
        print(f'Total Images: {index}')
        print(f'Total Images Correct Predicted: {correct_count}')       
    
        
print(f'Total Images: {len(validid_mapping)}')
print(f'Total Images Correct Predicted: {correct_count}')
    

Total Images: 1
Total Images Correct Predicted: 0
Total Images: 2
Total Images Correct Predicted: 0
Total Images: 3
Total Images Correct Predicted: 1
Total Images: 4
Total Images Correct Predicted: 2
Total Images: 5
Total Images Correct Predicted: 3
Total Images: 6
Total Images Correct Predicted: 4
Total Images: 7
Total Images Correct Predicted: 4
Total Images: 8
Total Images Correct Predicted: 5
Total Images: 9
Total Images Correct Predicted: 6
Total Images: 10
Total Images Correct Predicted: 6
Total Images: 11
Total Images Correct Predicted: 7
Total Images: 12
Total Images Correct Predicted: 7
Total Images: 13
Total Images Correct Predicted: 8
Total Images: 14
Total Images Correct Predicted: 8
Total Images: 15
Total Images Correct Predicted: 8
Total Images: 16
Total Images Correct Predicted: 9
Total Images: 17
Total Images Correct Predicted: 9
Total Images: 18
Total Images Correct Predicted: 9
Total Images: 19
Total Images Correct Predicted: 10
Total Images: 20
Total Images Correct P

In [100]:
validid_mapping['4c224e64ad49dade']

47378

In [101]:
predicted_label, good_matches_sift, good_matches_orb = tf_predictor(
        trainid_mapping, trainid_sift_des, trainid_sift_des,
        validid_sift_des['4c224e64ad49dade'], validid_sift_des['4c224e64ad49dade']
    )

Referenced : 25 training images
Referenced : 50 training images
Referenced : 75 training images
Referenced : 100 training images
Referenced : 125 training images
Referenced : 150 training images
Referenced : 175 training images
Referenced : 200 training images
Referenced : 225 training images
Referenced : 250 training images


In [104]:
good_matches_sift

{10419: 19, 47378: 38, 168098: 21, 162833: 70, 1924: 12}

In [105]:
predicted_label

162833

In [96]:
matches = matching(validid_orb_des['03e1331890523d51'], validid_orb_des['4c224e64ad49dade'])

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\flann\src\miniflann.cpp:336: error: (-210:Unsupported format or combination of formats) in function 'cv::flann::buildIndex_'
> type=0
> 

In [89]:
c = good_matching(matches, 0.5)
print(c)

0


In [90]:
matches = matching(validid_sift_des['03e1331890523d51'], trainid_sift_des['2a17c380e2f32622'])

In [91]:
c = good_matching(matches, 0.5)
print(c)

0


In [98]:
validid_sift_des['03e1331890523d51']

array([[23.,  3.,  0., ...,  0.,  0.,  0.],
       [ 5., 16.,  0., ...,  0.,  0.,  0.],
       [50.,  0.,  0., ...,  3.,  2.,  1.],
       ...,
       [37., 24.,  1., ..., 49., 37.,  4.],
       [32., 60., 25., ..., 17.,  0.,  0.],
       [ 0.,  0.,  0., ..., 20.,  0.,  0.]], dtype=float32)